In [ ]:
!pip3 install contextily

In [ ]:
import geopandas as gpd
import contextily as ctx
from sklearn.datasets import fetch_california_housing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# データ読み込み
cal = fetch_california_housing(as_frame=True)
df = cal.frame.copy()

print(df)

In [ ]:
# GeoDataFrameを作成（経度lon, 緯度lat）
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["Longitude"], df["Latitude"]),
    crs="EPSG:4326"  # WGS84 (経緯度)
).to_crs(epsg=3857)  # Web Mercatorに変換 (タイルと整合)

# プロット
fig, ax = plt.subplots(figsize=(8, 8))
gdf.plot(
    ax=ax,
    column="MedHouseVal",
    cmap="plasma",
    markersize=10,
    alpha=0.6,
    legend=True
)

# 背景にOpenStreetMapタイルを追加（白地図風）
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)
ax.set_axis_off()
ax.set_title("California Housing Prices", fontsize=14)

plt.show()

In [ ]:
# 0) データ読み込み
cal = fetch_california_housing(as_frame=True)
dfX = cal.data.copy()
y = cal.target.to_frame(name="MedHouseVal").to_numpy()  # (n,1)

# 1) 計画行列 X （切片を先頭に追加）
X = pd.concat([pd.Series(1.0, index=dfX.index, name="(intercept)"), dfX], axis=1)

# 2) テスト5サンプルを確保
rng = np.random.default_rng(42)
idx = np.arange(len(X))
rng.shuffle(idx)
test_idx = idx[:5]
train_idx = idx[5:]

X_train = X.iloc[train_idx].to_numpy()   # (n_train, p)
y_train = y[train_idx]                   # (n_train, 1)
X_test  = X.iloc[test_idx].to_numpy()    # (5, p)
y_test  = y[test_idx]                    # (5, 1)

# 3) β = (X^T X)^{-1} X^T y
XtX = X_train.T @ X_train                # (p, p)
XtX_inv = np.linalg.inv(XtX)             # (p, p)
Xty = X_train.T @ y_train                # (p, 1)
beta = XtX_inv @ Xty                     # (p, 1)

# 4) 学習残差 ε とテスト検証
y_hat_train = X_train @ beta
epsilon = y_train - y_hat_train

y_hat_test = X_test @ beta
test_errors = y_test - y_hat_test

# 5) 指標（テスト5件）
mae  = float(np.mean(np.abs(test_errors)))
rmse = float(np.sqrt(np.mean(test_errors**2)))
ss_res = float(np.sum((y_test - y_hat_test)**2))
ss_tot = float(np.sum((y_test - np.mean(y_test))**2))
r2 = 1.0 - ss_res / ss_tot if ss_tot != 0 else np.nan

# 6) 見やすく表示
coef = pd.DataFrame(beta, index=X.columns, columns=["beta"])
test_df = pd.DataFrame({
    "y_true":  y_test.flatten(),
    "y_pred":  y_hat_test.flatten(),
    "error":   test_errors.flatten()
}, index=pd.Index(test_idx, name="row_id")).sort_index()

print("=== beta (回帰係数) ===")
print(coef)

print("\n=== 学習残差 epsilon の統計 ===")
print(pd.Series({
    "mean(epsilon)": float(np.mean(epsilon)),
    "std(epsilon)":  float(np.std(epsilon))
}))

print("\n=== テスト5件の検証 ===")
print(test_df)

print("\nMAE (test):  {:.6f}".format(mae))
print("RMSE (test): {:.6f}".format(rmse))
print("R^2  (test): {:.6f}".format(r2))
